In [362]:
def parse_pullrequest(event):
    if event.get('repo') and event['actor'].get('login'):
        pullreq_id=event['id']
        repoid=event['repo']['id']
        user=event['actor']['login']
        if event['payload']['action']=='closed':
            closed_at=date_from_json(event['created_at'])
        else:
            closed_at=''
            merged_at=''
        if event['payload']['action']=='opened':
            opened_at=date_from_json(event['created_at'])
            merged_at=''
        else:
            opened_at=''
            merged_at=''
        return (pullreq_id, repoid, user, opened_at, closed_at, merged_at)
    elif event.get('repository') and event['payload'].get('actor'):
        pullreq_id=event['id']
        repoid=event['repository']['id']
        user=event['payload']['actor']
        opened_at=date_from_json(event.get('created_at'))
        if event['payload']['action']=='closed':
            closed_at=date_from_json(event['payload']['pull_request'].get('closed_at'))
        else:
            closed_at=''
        if event['payload']['pull_request']['merged_at']==True:
            merged_at=date_from_json(event['payload']['pull_request'].get('merged_at'))
        else:
            merged_at=''
        return (pullreq_id, repoid, user, opened_at, closed_at, merged_at)
    elif event.get('repository'):
        pullreq_id=event['payload']['pull_request']['id']
        repoid=event['repository']['id']
        user=event['actor']
        opened_at=date_from_json(event.get('created_at'))
        if event['payload']['action']=='closed':
            closed_at=date_from_json(event['payload']['pull_request'].get('closed_at'))
        else:
            closed_at='' 
        if event['payload']['pull_request']['merged_at']==True:
            merged_at=date_from_json(event['payload']['pull_request'].get('merged_at'))
        else:
            merged_at=''
        return (pullreq_id, repoid, user, opened_at, closed_at, merged_at)

In [321]:
#!/usr/bin/env python
# encoding: utf-8

import sys
import os
import math
import pprint
import json
import sqlite3
import time
import datetime
import numpy as np
from os import listdir
from os.path import isfile, join

#dbpath = '/Users/mszell/data/githubarchive/derived/'
#dbname = 'githubarchive.db'
#datapath = '/Users/mszell/data/githubarchive/raw/'

dbpath = ''
dbname = 'REPO_TEST4.sqlite'
datapath = os.getcwd()


In [322]:
def date_from_filename(fname):
    dt = datetime.datetime.strptime(fname[len('githubarchive_'):-len('.json')], '%Y-%m-%d')
    return dt.strftime('%Y-%m-%d')

In [323]:
def date_from_json(creation_date):
    dt = datetime.datetime.strptime(creation_date[:19], '%Y-%m-%dT%H:%M:%S')
    return dt

In [324]:

"""Parse all data from githubarchive (daily files) to build a new sqlite db

The database contains the following tables:
repository: (repositoryid INTEGER PRIMARY KEY, repositoryname TEXT, userid TEXT, created_at INTEGER, language TEXT, description TEXT, url TEXT, fork INTEGER)
push: (repositoryid INTEGER PRIMARY KEY, pushed_at DATE, userid TEXT)
watch: (repositoryid INTEGER PRIMARY KEY, watched_at DATE,  userid TEXT, watchers INTEGER)
memberadd: (repositoryid INTEGER PRIMARY KEY, added_at DATE, repositoryid INTEGER, userid INTEGER, byuserid INTEGER)
follow: (repositoryid INTEGER PRIMARY KEY, followed_at DATE, targetuserid TEXT, followers INTEGER, repos  byuserid TEXT)
star:()

The tables contain data relevant only to those repositories which have been created in the timespan that the files cover. Exception is table follows which contains all follow events.

Note: This script will only work with githubarchive data starting at 2012-09-22. Reason: Before this time, the repositoryid does not exist for PushEvents.


Parameters
----------
see below
dbpath: string
datapath: string

Dependencies
------------
see imports below

Returns
-------
n/a

Example
-------
>>> python build_all.py

See Also
--------


Created on  2016-10-27
Last update 2016-10-27
"""

'Parse all data from githubarchive (daily files) to build a new sqlite db\n\nThe database contains the following tables:\nrepository: (repositoryid INTEGER PRIMARY KEY, repositoryname TEXT, userid TEXT, created_at INTEGER, language TEXT, description TEXT, url TEXT, fork INTEGER)\npush: (repositoryid INTEGER PRIMARY KEY, pushed_at DATE, userid TEXT)\nwatch: (repositoryid INTEGER PRIMARY KEY, watched_at DATE,  userid TEXT, watchers INTEGER)\nmemberadd: (repositoryid INTEGER PRIMARY KEY, added_at DATE, repositoryid INTEGER, userid INTEGER, byuserid INTEGER)\nfollow: (repositoryid INTEGER PRIMARY KEY, followed_at DATE, targetuserid TEXT, followers INTEGER, repos  byuserid TEXT)\nstar:()\n\nThe tables contain data relevant only to those repositories which have been created in the timespan that the files cover. Exception is table follows which contains all follow events.\n\nNote: This script will only work with githubarchive data starting at 2012-09-22. Reason: Before this time, the reposito

In [414]:
def main():
    #build_repository()
    #build_push()
    #build_watch()
    #build_pull_request_opened()
    #build_pull_request_closed()
    #build_pull_request_merged()
    #build_fork()
    #build_star()
    #build_memberadd()
    #build_membership()
    #build_follow()

In [326]:
def parse_create_event(event):
    if event['payload'].get('object')=='repository' and event['repo'].get('id') and event['payload'].get('name') and event.get('actor').get('login'):
        repo_name=event['payload'].get('name')
        repoid=event['repo']['id']
        created_at=date_from_json(event.get('created_at'))
        user=event['actor']['login']
        description='NA'
        #print repoid, repo_name, user, created_at, description
        return repoid, repo_name, user, created_at, description
    elif event.get('payload') and event['payload'].get('ref_type')=='repository' and event.get('repository') and event.get('actor'):
        repo_name=event['repository']['name']
        repoid=event['repository']['id']
        created_at=date_from_json(event['repository'].get('created_at'))
        user=event['actor']
        description=event['payload'].get('description')
        return repoid, repo_name, user, created_at, description
    elif event.get('payload') and event['payload'].get('ref_type')=='repository' and event.get('repo') and event.get('actor'):
        repo_name=event['repo']['name']
        repoid=event['repo']['id']
        created_at=date_from_json(event.get('created_at'))
        user=event['actor']['login']
        description=event['payload'].get('description')
        return repoid, repo_name, user, created_at, description

In [359]:
def parse(event):
    #print event['type']
    if type(event)==dict:
        if event.get('repo') and event['repo'].get('id'):
            d=date_from_json(event.get('created_at'))
            if event.get('user'):
                if event['user']['type']=='User':
                    u=event['user']['login']
                    p=event['repo']['id']
                    return (u, p, d)
            elif event.get('actor'):
                if 'login' in event['actor'].keys():
                    u=event['actor']['login']
                    p=event['repo']['id']
                    return (u, p, d)
                elif event.get('payload') and event['payload'].get('actor'):
                    u=event['payload']['actor']
                    p=event['repo']['id']
                    return (u, p, d)
        elif event.get('repository'):
            u=event['actor']
            p=event['repository']['id']
            d=date_from_json(event.get('created_at'))
            return (u, p, d)

In [376]:
def parse_pullrequest_opened(event):
    if event.get('repo') and event['actor'].get('login'):
        pullreq_id=event['id']
        repoid=event['repo']['id']
        user=event['actor']['login']
        if event['payload']['action']=='opened':
            opened_at=date_from_json(event['created_at'])
        return (pullreq_id, repoid, user, opened_at)
    elif event.get('repository') and event['payload'].get('actor'):
        pullreq_id=event['id']
        repoid=event['repository']['id']
        user=event['payload']['actor']
        opened_at=date_from_json(event.get('created_at'))
        return (pullreq_id, repoid, user, opened_at)
    elif event.get('repository'):
        pullreq_id=event['payload']['pull_request']['id']
        repoid=event['repository']['id']
        user=event['actor']
        opened_at=date_from_json(event.get('created_at'))
        return (pullreq_id, repoid, user, opened_at)

In [390]:
def parse_pullrequest_closed(event):
    if event.get('repo') and event['actor'].get('login'):
        pullreq_id=event['id']
        repoid=event['repo']['id']
        user=event['actor']['login']
        closed_at=date_from_json(event['created_at'])
        return (pullreq_id, repoid, user, closed_at)
    elif event.get('repository') and event['payload'].get('actor'):
        pullreq_id=event['id']
        repoid=event['repository']['id']
        user=event['payload']['actor']
        closed_at=date_from_json(event['payload']['pull_request'].get('closed_at'))
        return (pullreq_id, repoid, user, closed_at)
    elif event.get('repository'):
        pullreq_id=event['payload']['pull_request']['id']
        repoid=event['repository']['id']
        user=event['actor']
        closed_at=date_from_json(event['payload']['pull_request'].get('closed_at'))
        return (pullreq_id, repoid, user, closed_at)

In [435]:
def parse_pullrequest_merged(event):
    if event.get('repository') and event['payload'].get('actor'):
        pullreq_id=event['id']
        repoid=event['repository']['id']
        user=event['payload']['actor']
        merged_at=date_from_json(event['payload']['pull_request'].get('closed_at'))
        #print pullreq_id, repoid, user, merged_at
        return (pullreq_id, repoid, user, merged_at)
    elif event.get('repo'):
        pullreq_id=event['payload']['pull_request']['id']
        repoid=event['repo']['id']
        user=event['actor']['login']
        merged_at=date_from_json(event['payload']['pull_request'].get('closed_at'))
        #print pullreq_id, repoid, user, merged_at
        return (pullreq_id, repoid, user, merged_at)

In [328]:
def build_repository():
    # Create table
    conn = sqlite3.connect(dbpath + dbname)
    c = conn.cursor()
    c.execute('''DROP TABLE IF EXISTS repository''')
    c.execute('''CREATE TABLE IF NOT EXISTS repository
                 (repositoryid INTEGER PRIMARY KEY, repositoryname TEXT, userid TEXT, created_at DATE, description TEXT)''')
    conn.commit()
    conn.close()

    # Parse files
    #filenames = [ f for f in listdir(datapath) if isfile(join(datapath,f)) ]
    filenames=sorted([i for i in os.listdir(datapath) if i.startswith("githubarchive_") and i.endswith('json')])
    for filename in filenames:
        conn = sqlite3.connect(dbpath + dbname)
        c = conn.cursor()
        i = 0
        with open(filename) as jsonfile:
            for line in jsonfile:
                try: 
                    jsonline = json.loads(unicode(line, errors='ignore').strip())
                    #print jsonline.keys()
                    if jsonline.get("type") == "CreateEvent" and jsonline.get('payload').get('object')=='repository':
                        try:
                            repoid, repo_name, user, created_at, description=parse_create_event(jsonline)
                            query = 'INSERT OR REPLACE INTO repository VALUES (?,?,?,?,?)'
                            c.execute(query, (repoid, repo_name, user, created_at, description))
                            i = i+1
                            #print i
                        except: # Rarely, JSON is not valid even if I tryy super hard
                            pass
                    elif jsonline.get("type") == "CreateEvent" and jsonline.get('payload').get('ref_type')=='repository':
                        repoid, repo_name, user, created_at, description=parse_create_event(jsonline)
                        query = 'INSERT OR REPLACE INTO repository VALUES (?,?,?,?,?)'
                        c.execute(query, (repoid, repo_name, user, created_at, description))
                        i = i+1
                except:
                    for k in ['{"repo"'+l for l in line.split('{"repo"')[1:]]:
                        jsonline=json.loads(unicode(k, errors='ignore').strip())
                        last_event=jsonline
                        if jsonline.get("type") == "CreateEvent" and jsonline.get('payload').get('object')=='repository':
                            try:
                                repoid, repo_name, user, created_at, description=parse_create_event(jsonline)
                                query = 'INSERT OR REPLACE INTO repository VALUES (?,?,?,?,?)'
                                c.execute(query, (repoid, repo_name, user, created_at, description))
                                i = i+1
                            except:
                                pass
                        elif jsonline.get("type") == "CreateEvent" and jsonline['payload'].get('ref_type')=='repository':
                            repoid, repo_name, user, created_at, description=parse_create_event(jsonline)
                            query = 'INSERT OR REPLACE INTO repository VALUES (?,?,?,?,?)'
                            c.execute(query, (repoid, repo_name, user, created_at, description))
                            i = i+1
            print "Done file " + filename + ". Found and inserted " + str(i) + " valid CreationEvents."
        conn.commit()
        conn.close()

In [330]:
def build_watch():
    # Create table
    conn = sqlite3.connect(dbpath + dbname)
    c = conn.cursor()
    c.execute('''DROP TABLE IF EXISTS watch''')
    c.execute('''CREATE TABLE IF NOT EXISTS watch
                 (repositoryid INTEGER, watched_at DATE, byuserlogin TEXT)''')
    conn.commit()
    conn.close()
    
    # Parse files
    #filenames = [ f for f in listdir(datapath) if isfile(join(datapath,f)) ]
    filenames=sorted([i for i in os.listdir(datapath) if i.startswith("githubarchive_") and i.endswith('json')])
    for filename in filenames:
        conn = sqlite3.connect(dbpath + dbname)
        c = conn.cursor()
        i = 0
        with open(filename) as jsonfile:
            for line in jsonfile:
                try: 
                    jsonline = json.loads(unicode(line, errors='ignore').strip())
                    #print jsonline.keys()
                    if jsonline.get("type") == "WatchEvent":
                        try:
                            user, project, date=parse(jsonline)
                            query = 'INSERT OR REPLACE INTO watch VALUES (?,?,?)'
                            c.execute(query, (project, date, user))
                            i = i+1
                        except: #repoid or user id missing
                            pass
                except:
                    for k in ['{"repo"'+l for l in line.split('{"repo"')[1:]]:
                        jsonline=json.loads(unicode(k, errors='ignore').strip())
                        last_event=jsonline
                        if jsonline.get("type") == "WatchEvent":
                            try:
                                user, project, date=parse(jsonline)
                                query = 'INSERT OR REPLACE INTO watch VALUES (?,?,?)'
                                c.execute(query, (project, date, user))
                                i = i+1
                            except: #repoid or user id missing
                                pass
        print "Done file " + filename + ". Found and inserted " + str(i) + " valid WatchEvents."
        conn.commit()
        conn.close()

In [329]:
def build_push():
    # Create table
    conn = sqlite3.connect(dbpath + dbname)
    c = conn.cursor()
    c.execute('''DROP TABLE IF EXISTS push''')
    c.execute('''CREATE TABLE IF NOT EXISTS push
                 (repositoryid INTEGER, pushed_at DATE, userlogin TEXT)''')
    conn.commit()
    conn.close()
    
    # Parse files
    #filenames = [ f for f in listdir(datapath) if isfile(join(datapath,f)) ]
    filenames=sorted([i for i in os.listdir(datapath) if i.startswith("githubarchive_") and i.endswith('json')])
    for filename in filenames:
        conn = sqlite3.connect(dbpath + dbname)
        c = conn.cursor()
        i = 0
        with open(filename) as jsonfile:
            for line in jsonfile:
                try: 
                    jsonline = json.loads(unicode(line, errors='ignore').strip())
                    #print jsonline.keys()
                    if jsonline.get("type") == "PushEvent":
                        try:
                            user, project, date=parse(jsonline)
                            query = 'INSERT OR REPLACE INTO push VALUES (?,?,?)'
                            c.execute(query, (project, date, user))
                            i = i+1
                        except: #repoid or user id missing
                            pass
                except:
                    for k in ['{"repo"'+l for l in line.split('{"repo"')[1:]]:
                        jsonline=json.loads(unicode(k, errors='ignore').strip())
                        last_event=jsonline
                        if jsonline.get("type") == "PushEvent":
                            try:
                                user, project, date=parse(jsonline)
                                query = 'INSERT OR REPLACE INTO push VALUES (?,?,?)'
                                c.execute(query, (project, date, user))
                                i = i+1
                            except: #repoid or user id missing
                                pass
        print "Done file " + filename + ". Found and inserted " + str(i) + " valid PushEvents."
        conn.commit()
        conn.close()

In [416]:
def build_pull_request_merged():
    # Create table
    conn = sqlite3.connect(dbpath + dbname)
    c = conn.cursor()
    c.execute('''DROP TABLE IF EXISTS pull_request_merged''')
    c.execute('''CREATE TABLE IF NOT EXISTS pull_request_merged
                 (pulrrequestid INTEGER PRIMARY KEY, repositoryid INTEGER, userlogin TEXT, merged_at DATE)''')
    conn.commit()
    conn.close()
    filenames=sorted([i for i in os.listdir(datapath) if i.startswith("githubarchive_") and i.endswith('json')])
    for filename in filenames:
        conn = sqlite3.connect(dbpath + dbname)
        c = conn.cursor()
        i = 0
        with open(filename) as jsonfile:
            for line in jsonfile:
                try: 
                    jsonline = json.loads(unicode(line, errors='ignore').strip())
                    #print jsonline.keys()
                    if jsonline.get("type") == "PullRequestEvent" and jsonline['payload']['pull_request'].get('merged')==True:
                        try:
                            pullreq_id, repoid, user, merged_at=parse_pullrequest_merged(jsonline)
                            query = 'INSERT INTO pull_request_merged VALUES (?,?,?,?)'
                            c.execute(query, (pullreq_id, repoid, user, merged_at))
                            i = i+1
                        except: #repoid or user id missing
                            pass
                except:
                    for k in ['{"repo"'+l for l in line.split('{"repo"')[1:]]:
                        jsonline=json.loads(unicode(k, errors='ignore').strip())
                        last_event=jsonline
                        if jsonline.get("type") == "PullRequestEvent" and jsonline['payload']['pull_request'].get('merged')==True:
                            try:
                                pullreq_id, repoid, user, merged_at =parse_pullrequest_merged(jsonline)
                                query = 'INSERT INTO pull_request_merged VALUES (?,?,?,?)'
                                c.execute(query, (pullreq_id, repoid, user, merged_at))
                                i = i+1
                            except: #repoid or user id missing
                                pass
        print "Done file " + filename + ". Found and inserted " + str(i) + " valid PullRequestMerged."
        conn.commit()
        conn.close()

In [385]:
def build_pull_request_opened():
    # Create table
    conn = sqlite3.connect(dbpath + dbname)
    c = conn.cursor()
    c.execute('''DROP TABLE IF EXISTS pull_request_opened''')
    c.execute('''CREATE TABLE IF NOT EXISTS pull_request_opened
                 (pulrrequestid INTEGER PRIMARY KEY, repositoryid INTEGER, userlogin TEXT, opened_at DATE)''')
    conn.commit()
    conn.close()
    filenames=sorted([i for i in os.listdir(datapath) if i.startswith("githubarchive_") and i.endswith('json')])
    for filename in filenames:
        conn = sqlite3.connect(dbpath + dbname)
        c = conn.cursor()
        i = 0
        with open(filename) as jsonfile:
            for line in jsonfile:
                try: 
                    jsonline = json.loads(unicode(line, errors='ignore').strip())
                    #print jsonline.keys()
                    if jsonline.get("type") == "PullRequestEvent" and jsonline['payload']['action']=='opened':
                        try:
                            pullreq_id, repoid, user, opened_at=parse_pullrequest_opened(jsonline)
                            query = 'INSERT INTO pull_request_opened VALUES (?,?,?,?)'
                            c.execute(query, (pullreq_id, repoid, user, opened_at))
                            i = i+1
                        except: #repoid or user id missing
                            pass
                except:
                    for k in ['{"repo"'+l for l in line.split('{"repo"')[1:]]:
                        jsonline=json.loads(unicode(k, errors='ignore').strip())
                        last_event=jsonline
                        if jsonline.get("type") == "PullRequestEvent" and jsonline['payload']['action']=='opened':
                            try:
                                pullreq_id, repoid, user, opened_at =parse_pullrequest_opened(jsonline)
                                query = 'INSERT INTO pull_request_opened VALUES (?,?,?,?)'
                                c.execute(query, (pullreq_id, repoid, user, opened_at))
                                i = i+1
                            except: #repoid or user id missing
                                pass
        print "Done file " + filename + ". Found and inserted " + str(i) + " valid PullRequestOpened."
        conn.commit()
        conn.close()

In [394]:
def build_pull_request_closed():
    # Create table
    conn = sqlite3.connect(dbpath + dbname)
    c = conn.cursor()
    c.execute('''DROP TABLE IF EXISTS pull_request_closed''')
    c.execute('''CREATE TABLE IF NOT EXISTS pull_request_closed
                 (pulrrequestid INTEGER PRIMARY KEY, repositoryid INTEGER, userlogin TEXT, closed_at DATE)''')
    conn.commit()
    conn.close()
    filenames=sorted([i for i in os.listdir(datapath) if i.startswith("githubarchive_") and i.endswith('json')])
    for filename in filenames:
        conn = sqlite3.connect(dbpath + dbname)
        c = conn.cursor()
        i = 0
        with open(filename) as jsonfile:
            for line in jsonfile:
                try: 
                    jsonline = json.loads(unicode(line, errors='ignore').strip())
                    #print jsonline.keys()
                    if jsonline.get("type") == "PullRequestEvent" and jsonline['payload']['action']=='closed':
                        try:
                            pullreq_id, repoid, user, closed_at=parse_pullrequest_closed(jsonline)
                            query = 'INSERT INTO pull_request_closed VALUES (?,?,?,?)'
                            c.execute(query, (pullreq_id, repoid, user, closed_at))
                            i = i+1
                        except: #repoid or user id missing
                            pass
                except:
                    for k in ['{"repo"'+l for l in line.split('{"repo"')[1:]]:
                        jsonline=json.loads(unicode(k, errors='ignore').strip())
                        last_event=jsonline
                        if jsonline.get("type") == "PullRequestEvent" and jsonline['payload']['action']=='closed':
                            try:
                                pullreq_id, repoid, user, closed_at =parse_pullrequest_closed(jsonline)
                                query = 'INSERT INTO pull_request_closed VALUES (?,?,?,?)'
                                c.execute(query, (pullreq_id, repoid, user, closed_at))
                                i = i+1
                            except: #repoid or user id missing
                                pass
        print "Done file " + filename + ". Found and inserted " + str(i) + " valid PullRequestClosed."
        conn.commit()
        conn.close()

In [ ]:
if __name__ == '__main__':
    main()

Done file githubarchive_2011-02-12.json. Found and inserted 0 valid PullRequestMerged.
Done file githubarchive_2011-02-13.json. Found and inserted 0 valid PullRequestMerged.
Done file githubarchive_2011-02-14.json. Found and inserted 0 valid PullRequestMerged.
Done file githubarchive_2011-03-14.json. Found and inserted 0 valid PullRequestMerged.
Done file githubarchive_2011-03-15.json. Found and inserted 0 valid PullRequestMerged.
Done file githubarchive_2011-03-28.json. Found and inserted 0 valid PullRequestMerged.
Done file githubarchive_2011-03-30.json. Found and inserted 0 valid PullRequestMerged.
Done file githubarchive_2011-04-06.json. Found and inserted 0 valid PullRequestMerged.
Done file githubarchive_2011-04-08.json. Found and inserted 0 valid PullRequestMerged.
Done file githubarchive_2013-02-27.json. Found and inserted 0 valid PullRequestMerged.


In [1]:
"""#TEST PARSER
fs=sorted([i for i in os.listdir(datapath) if i.startswith("githubarchive_2015") and i.endswith('json')])
for f in fs:
    with open(f) as jsonfile:
        for line in jsonfile:
            jsonline = json.loads(unicode(line, errors='ignore').strip())
            if jsonline.get("type") == "PullRequestEvent" and jsonline['payload']['pull_request']['merged']==True:
                x=jsonline
                parse_pullrequest_merged(jsonline)
                alma"""

'#TEST PARSER\nfs=sorted([i for i in os.listdir(datapath) if i.startswith("githubarchive_2015") and i.endswith(\'json\')])\nfor f in fs:\n    with open(f) as jsonfile:\n        for line in jsonfile:\n            jsonline = json.loads(unicode(line, errors=\'ignore\').strip())\n            if jsonline.get("type") == "PullRequestEvent" and jsonline[\'payload\'][\'pull_request\'][\'merged\']==True:\n                x=jsonline\n                parse_pullrequest_merged(jsonline)\n                alma'

## TODO

- add files asn a paramter
- join pullrequests
- write tests
- see parsing differences over years
- Write these:
    - build_fork()
    - build_star()
    - build_memberadd()
    - build_membership()
    - build_follow()